Import packages

In [1]:
import pandas as pd
import numpy as np
# for moving around libraries 
import os 
# for visualizing 
import matplotlib.pyplot as plt
import seaborn as sns
# for finding word 
import re

# NYC real-estate data

## Zillow housing data
### src: https://www.zillow.com/research/data/

Yearly Home values, by neighbourhood

In [2]:
# move to correct directory (NOTICE problems on git?)
os.chdir("../Data")

FileNotFoundError: [WinError 2] The system cannot find the file specified: '../Data'

In [3]:
home_values = pd.read_csv("./Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv", 
                     header=0, 
                     infer_datetime_format=True)
                     #names=['Month','airbnb office: (New York)']).iloc[1:]

FileNotFoundError: [Errno 2] File ./Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv does not exist: './Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv'

keep only NYC

In [4]:
ny_home_values=home_values.loc[home_values['City'] == 'New York']

NameError: name 'home_values' is not defined

drop unused columns

In [5]:
ny_home_values.reset_index(level=0, inplace=True)

NameError: name 'ny_home_values' is not defined

In [6]:
ny_home_values = ny_home_values.drop(columns=['RegionType','StateName','State',
                                             'City','index'])

NameError: name 'ny_home_values' is not defined

In [7]:
ny_home_values

NameError: name 'ny_home_values' is not defined

## Summaries of Neighborhood and Citywide Annual Sales sorted by borough and neighborhood.
### src: https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page

In [18]:
# move to correct directory (NOTICE problems on git?)
os.chdir("C:/Users/meita/Documents/GitHub/project-Karta/Data/sales_data")

In order to open the excel files we'll use the following library

In [12]:
!pip install xlrd

Check xls encoding value to be able to read it with python

# Extract, transform, load (ETL)

## Extract

Let's see all the sales data we collected

In [20]:
path = 'C:/Users/meita/Documents/GitHub/project-Karta/Data/sales_data'
files = os.listdir(path)
for f in files:
	print(f)

2009_bronx.xls
2009_brooklyn.xls
2009_manhattan.xls
2009_queens.xls
2009_statenisland.xls
2010_bronx.xls
2010_brooklyn.xls
2010_manhattan.xls
2010_queens.xls
2010_statenisland.xls
2011_bronx.xls
2011_brooklyn.xls
2011_manhattan.xls
2011_queens.xls
2011_statenisland.xls
2012_bronx.xls
2012_brooklyn.xls
2012_manhattan.xls
2012_queens.xls
2012_statenisland.xls
2013_bronx.xls
2013_brooklyn.xls
2013_manhattan.xls
2013_queens.xls
2013_statenisland.xls
2014_bronx.xls
2014_brooklyn.xls
2014_manhattan.xls
2014_queens.xls
2014_statenisland.xls
2015_bronx.xls
2015_brooklyn.xls
2015_manhattan.xls
2015_queens.xls
2015_statenisland.xls
2016_bronx.xls
2016_brooklyn.xls
2016_manhattan.xls
2016_queens.xls
2016_statenisland.xls
2017_bronx.xls
2017_brooklyn.xls
2017_manhattan.xls
2017_queens.xls
2017_statenisland.xls
2018_bronx.xlsx
2018_brooklyn.xlsx
2018_manhattan.xlsx
2018_queens.xlsx
2018_statenisland.xlsx
2019_bronx.xlsx
2019_brooklyn.xlsx
2019_manhattan.xlsx
2019_queens.xlsx
2019_statenisland.xlsx


We can assume that the way the name of the file is formetted has an impact on the way the data is stractured inside (e.g sales_brooklyn_03 is probably different than 2017_brooklyn but not as different then sales_brooklyn_05.


To select only certain files that we assume would be alike, we'll import glob

In [22]:
import glob

### 2017

Let's save all files from 2017 to the same list

In [25]:
sls_2017 = [file for file in glob.glob("2017_*")]
print(sls_2017)

['2017_bronx.xls', '2017_brooklyn.xls', '2017_manhattan.xls', '2017_queens.xls', '2017_statenisland.xls']


Because we are dealing with excel files (and not csv for example) we need to make sure the file encoding is fitting the way that we open the file.

In [74]:
def df_encoding(file):
    with open(f"./{file}") as f:
        return(f.encoding)
df_encoding(sls_2017[0])

'cp65001'

Giving each DataFrame variable a standertized name

In [61]:
def df_var_name(file):
    file_name = file
    file_var_name = "ann_sales_{tmp_file_name}" 
    return(file_var_name.format(tmp_file_name = file_name))
df_var_name(sls_2017[0])

'ann_sales_2017_bronx.xls'

Taking a file name and returning a DataFrame 

In [78]:
def df_extract(file_name):
    file = file_name
    x = pd.read_excel(f"./{file}", 
                     header=0, 
                     infer_datetime_format=True,
                     encoding=df_encoding(file_name))
    return(x)

Take a list of excel files and return a dictonary containing each file as a data frame 

In [242]:
def df_import(lst_of_files):
        df_dict = {df_var_name(name): df_extract(name) for name in lst_of_files}
        return(df_dict)

For each year, create a dictionnary containing the a DataFram for each Borough in NYC

In [259]:
dict_sls_2017 = df_import(sls_2017)

2017_bronx.xls
2017_brooklyn.xls
2017_manhattan.xls
2017_queens.xls
2017_statenisland.xls


## Clean

See the keys in a dictionary 

In [260]:
dict_sls_2017.keys()

dict_keys(['ann_sales_2017_bronx.xls', 'ann_sales_2017_brooklyn.xls', 'ann_sales_2017_manhattan.xls', 'ann_sales_2017_queens.xls', 'ann_sales_2017_statenisland.xls'])

### 2017 Format

Takes a key and dictionary of DF's and returns single DataFrame (value of key)

In [261]:
def df_2017_std_return_value(key, dicto): 
    return(dicto.get(key))

Takes a single dataFrame and cleans it according to the 2017 excel file standart 

In [262]:
def df_cln_2017_std(df_to_cln): 
    df_clnd = df_to_cln # copy the dataFrame we want to clean into a new DF
    df_clnd = df_clnd.drop(df_clnd.index[:3]) # drop the first three rows
    new_header = df_clnd.iloc[0] #grab the first row for the header
    df_clnd = df_clnd[1:] #take the data less the header row
    df_clnd.columns = new_header #set the header row as the df header
    df_clnd = df_clnd.reset_index(level=0, inplace=False) # reset the index (after removing the first - now 4 - rows)
    df_clnd = df_clnd.drop(columns=['index']) # drop the column with the name "index" (which is the old index before reset)
    df_clnd = df_clnd.rename_axis('', axis='columns') # remove row number falsely used as column name
    return(df_clnd)

Takes a dictionary of 'dirty' DataFrames and return the dictionary cleaned 

In [263]:
def dct_of_df_clnr_2017(dict_to_cln):
    return ({key: df_cln_2017_std(df_2017_std_return_value(key, dict_to_cln)) for key in (list(dict_to_cln.keys()))})
dict_sls_2017.update(dct_of_df_clnr_2017(dict_sls_2017))

In [267]:
dict_sls_2017.get('ann_sales_2017_manhattan.xls')

,BOROUGH\n,NEIGHBORHOOD\n,BUILDING CLASS CATEGORY\n,TAX CLASS AS OF FINAL ROLL 17/18,BLOCK\n,LOT\n,EASE-MENT\n,BUILDING CLASS AS OF FINAL ROLL 17/18,ADDRESS\n,APARTMENT NUMBER\n,...,RESIDENTIAL UNITS\n,COMMERCIAL UNITS\n,TOTAL UNITS\n,LAND SQUARE FEET\n,GROSS SQUARE FEET\n,YEAR BUILT\n,TAX CLASS AT TIME OF SALE\n,BUILDING CLASS AT TIME OF SALE\n,SALE PRICE\n,SALE DATE\n
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,375,28,,C4,738 EAST 6TH STREET,,...,11,0,11,1750,6500,1900,2,C4,3750000,2017-04-03 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,36,,C7,27 AVENUE C,,...,24,1,25,2650,9960,1910,2,C7,5235000,2017-07-11 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,5,,C2,151 AVENUE B,,...,5,0,5,2139,4416,1900,2,C2,0,2017-06-16 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,400,61,,C7,193 EAST 4TH STREET,,...,11,1,12,2388,8045,1900,2,C7,7675000,2017-11-13 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18637,1,WASHINGTON HEIGHTS UPPER,32 HOSPITAL AND HEALTH FACILITIES,4,2152,47,,I9,2414 AMSTERDAM AVENUE,,...,0,3,3,2500,3350,1902,4,I9,1375000,2017-12-08 00:00:00
18638,1,WASHINGTON HEIGHTS UPPER,33 EDUCATIONAL FACILITIES,4,2149,23,,W2,2495 AMSTERDAM AVENUE,,...,0,1,1,29775,253942,1969,4,W2,0,2017-04-21 00:00:00
18639,1,WASHINGTON HEIGHTS UPPER,33 EDUCATIONAL FACILITIES,4,2156,48,,W6,2520 AMSTERDAM AVENUE,,...,0,1,1,25875,140260,1967,4,W6,0,2017-04-21 00:00:00
18640,1,WASHINGTON HEIGHTS UPPER,33 EDUCATIONAL FACILITIES,4,2156,61,,W6,515 WEST 185 STREET,,...,0,1,1,2700,16262,2007,4,W6,0,2017-08-23 00:00:00


Dropping the first three rows

In [177]:
ann_sales_2017_bronx = ann_sales_2017_bronx.drop(ann_sales_2017_bronx.index[:3])

Turn the first row into the header